In [ ]:
# IMPORTANT:
# First of all, copy the 'use_this_in_the_tensorflow_code.zip' file into this environment.
# You have to do this every time you are running this in Colab, because it discards everything after you close it.

import os
import zipfile


try:
  # Open the file
  the_zip_file = zipfile.ZipFile('./use_this_in_the_tensorflow_code.zip', 'r')
  # Extract the contents to the same directory as this code is running in.
  the_zip_file.extractall('./')
  # Close the file
  the_zip_file.close()
except:
  print("The zip file could not be opened. Make sure you upload it to the same directory where this code is.")


In [ ]:
# Here, we define our directories. All manual, for clarity.

# Training data set

training_source_path = './fridgie_data_training'

# Check a condition
train_dir_check_airflow = os.path.join(training_source_path + '/check_airflow')

try:
  print(f"There are {len(os.listdir(train_dir_check_airflow))} images in the training data check_airflow.")
except:
  print("Looks like the training data wasn't extracted properly.")

# Testing data set

testing_source_path = './fridgie_data_testing'

test_dir_high_heatload = os.path.join(testing_source_path + '/high_heatload')

try:
  print(f"There are {len(os.listdir(test_dir_high_heatload))} images in the testing data high_heatload.")
except:
  print("Looks like the testing data wasn't extracted properly.")



There are 10000 images in the training data check_airflow.
There are 2500 images in the testing data high_heatload.


In [ ]:
# Actual machine learing thing: the neural network we have

import tensorflow as tf

# This is the model definition

model = tf.keras.models.Sequential([
    # Since our images are 5-by-1 pixels, this can be fairly simple.
    tf.keras.layers.Flatten(), # Input layer. This is not really a layer, but it strips extra dimensions from the data
    tf.keras.layers.Dense(94, activation = tf.nn.relu), # Hidden layer
    tf.keras.layers.Dense(6, activation = tf.nn.softmax) # Output layer, must match the number of labels
])

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

In [ ]:
# Since we are reducing the temperature and duty cycle measurements as image data, we will use the image data generator.

from tensorflow.keras.preprocessing.image import ImageDataGenerator

no_of_batches = 32

train_gen = ImageDataGenerator( rescale = 1.0/255. ) # Image data is encoded as 8-bit values.

train_generator = train_gen.flow_from_directory(
    training_source_path,
    batch_size = no_of_batches
)

test_gen = ImageDataGenerator( rescale = 1.0/255. ) # Same as above

test_generator = test_gen.flow_from_directory(
    testing_source_path,
    batch_size = no_of_batches
)



Found 60000 images belonging to 6 classes.
Found 15000 images belonging to 6 classes.


In [ ]:
# Since training can be a long thing, it is worthwhile to do a callback function
# This stops training when we reached the required accuracy when training and validating the data
# Also, training for too long may have a negative effect on performance

class are_we_there_yet(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    # We can separate the training and validation accuracy.
    required_training_accuracy = 0.99
    required_validation_accuracy = 0.99
    # This runs at every epoch's end, and checks whether we can stop.
    if(logs.get('accuracy') >= required_training_accuracy and logs.get('val_accuracy') >= required_validation_accuracy):
      print("\n\nWe got it, stopping training.\n\n")
      self.model.stop_training = True

In [ ]:
# Now we do the training

no_of_epochs = 50 # This is way too many, the callback function will stop training when we are done.

history = model.fit(
    train_generator,
    epochs = no_of_epochs,
    verbose = 1,
    validation_data = test_generator,
    callbacks = [are_we_there_yet()]
)

Epoch 1/50
1875/1875 [==============================] - 112s 59ms/step - loss: 4.6720 - accuracy: 0.9096 - val_loss: 1.2648 - val_accuracy: 0.9216
Epoch 2/50
1875/1875 [==============================] - 105s 56ms/step - loss: 0.4529 - accuracy: 0.9769 - val_loss: 0.0579 - val_accuracy: 0.9898
Epoch 3/50
1875/1875 [==============================] - 100s 53ms/step - loss: 0.2537 - accuracy: 0.9832 - val_loss: 0.0561 - val_accuracy: 0.9923
Epoch 4/50
1875/1875 [==============================] - 101s 54ms/step - loss: 0.1980 - accuracy: 0.9847 - val_loss: 0.0472 - val_accuracy: 0.9907
Epoch 5/50
1875/1875 [==============================] - 102s 55ms/step - loss: 0.1474 - accuracy: 0.9869 - val_loss: 0.0287 - val_accuracy: 0.9953
Epoch 6/50
1875/1875 [==============================] - 101s 54ms/step - loss: 0.1332 - accuracy: 0.9873 - val_loss: 0.0712 - val_accuracy: 0.9902
Epoch 7/50
1875/1875 [==============================] - 101s 54ms/step - loss: 0.1297 - accuracy: 0.9879 - val_loss: 0

In [ ]:
# Save the model

model.save_weights('./saved_model') # Save the file in the same working directory.

# ...and at this point, you can process the model further, as you wish.

